# Hurricane Season Analysis

## Data Gathering

### Pseudocode
1.  import dependencies
2.  get csv file
3.  read csv file into pandas
4.  display
5.  cleanup/delete unnecessary columns

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import csv
file = "hurricane_data.csv"

In [2]:
# Add headerrow containing storm id and storm name to data rows and create list
hurricane_data = []

storm_id = "0000000"
storm_name = "UNNAMED"
with open(file) as hurricane_file:
    hurricane_reader = csv.reader(hurricane_file)
    next(hurricane_reader, None) 
    for row in hurricane_reader:
                if 'AL' in row[0]: 
                    storm_id = row[0].strip()
                    storm_name = row[1].strip()
#                     print(f'{storm_id}: {storm_name}')
                else:
                    oldformat = row[0]+row[1]
                    datetimeobject = datetime.strptime(oldformat,'%Y%m%d %H%M%S')
                    date = datetimeobject.strftime('%m-%d-%Y %H%M%S')
                    year = row[0][:4]
                    landfall = row[2].strip()
                    status = row[3].strip()
                    latitude = row[4].strip()
                    longitude = row[5].strip()
                    wind = row[6].strip()
                    pressure = row[7].strip()
                hurricane_data.append(
                    {
                        "Storm_Id": storm_id, 
                        "Name": storm_name,
                        "Date": date,
                        "Year": year,
                        "Landfall": landfall,
                        "Status" : status,
                        "Latitude" : latitude,
                        "Longitude" : longitude,
                        "Windspeed" : wind,
                        "Pressure" : pressure
                    }
                )    
                           
    

In [3]:
# Create Dataframe and reorder columns
hurricane_pd = pd.DataFrame(hurricane_data)
hurricane_pd = hurricane_pd.loc[:, ["Storm_Id", "Name", "Date", "Year", "Status", "Latitude",
                                    "Longitude", "Windspeed", "Pressure", "Landfall"]]
hurricane_pd.head()

,Storm_Id,Name,Date,Year,Status,Latitude,Longitude,Windspeed,Pressure,Landfall
0,0000000,UNNAMED,06-25-1851 000000,1851,HU,28.0N,94.8W,80,-999,
1,0000000,UNNAMED,06-25-1851 060000,1851,HU,28.0N,95.4W,80,-999,
2,0000000,UNNAMED,06-25-1851 120000,1851,HU,28.0N,96.0W,80,-999,
3,0000000,UNNAMED,06-25-1851 180000,1851,HU,28.1N,96.5W,80,-999,
4,0000000,UNNAMED,06-25-1851 210000,1851,HU,28.2N,96.8W,80,-999,L


# Data Cleanup

### Pseudocode
1. Check column counts for missing data
2. Check/Change column types for type conversion
3. Limit data to only the past 15 years, only Hurricanes and Tropical Storms, and only named storms
4. Get min date (when storm became Tropical Storm) and Max Date (When storm is no longer a tropical storm)
5. Calculate the duration of storm
6. Find nearest city for storms tha reached landfall
7. Get max windspeed row and return all columns
8. Categorize storms according to Saffir-Simpson scale https://en.wikipedia.org/wiki/Maximum_sustained_wind 

In [4]:
#Check column counts for missing data
hurricane_pd.count()

Storm_Id     52150
Name         52150
Date         52150
Year         52150
Status       52150
Latitude     52150
Longitude    52150
Windspeed    52150
Pressure     52150
Landfall     52150
dtype: int64

In [5]:
#Check column types for needed type conversions
hurricane_pd.dtypes

Storm_Id     object
Name         object
Date         object
Year         object
Status       object
Latitude     object
Longitude    object
Windspeed    object
Pressure     object
Landfall     object
dtype: object

In [6]:
#Change types for Windspeed and Pressure to numerice and Date to Datetime
hurricane_pd['Windspeed'] = pd.to_numeric(hurricane_pd['Windspeed'])
hurricane_pd['Pressure'] = pd.to_numeric(hurricane_pd['Pressure'])
hurricane_pd['Date'] = pd.to_datetime(hurricane_pd['Date'])
hurricane_pd.dtypes

Storm_Id             object
Name                 object
Date         datetime64[ns]
Year                 object
Status               object
Latitude             object
Longitude            object
Windspeed             int64
Pressure              int64
Landfall             object
dtype: object

In [19]:
#Limit data to only the past 15 years, 
# only Hurricanes and Tropical Storms, and 
# only named storms

hurricane_df_clean = hurricane_pd.loc[hurricane_pd["Year"] > "2002"]
hurricane_df_clean = hurricane_df_clean.loc[hurricane_df_clean['Status'].isin(['TS','HU'])]
hurricane_df_clean = hurricane_df_clean.loc[hurricane_df_clean['Name'] != "UNNAMED"]
hurricane_df_clean.head()

,Storm_Id,Name,Date,Year,Status,Latitude,Longitude,Windspeed,Pressure,Landfall
44120,AL012003,ANA,2003-04-21 00:00:00,2003,TS,31.4N,66.4W,50,995,
44121,AL012003,ANA,2003-04-21 06:00:00,2003,TS,30.6N,65.8W,50,995,
44122,AL012003,ANA,2003-04-21 12:00:00,2003,TS,30.0N,64.8W,50,995,
44123,AL012003,ANA,2003-04-21 18:00:00,2003,TS,29.4N,63.1W,50,994,
44124,AL012003,ANA,2003-04-22 00:00:00,2003,TS,29.4N,61.5W,50,994,


In [8]:
#Get min date (when storm became Tropical Storm) and Max Date (When storm is no longer a tropical storm)
#Calculate the duration

storm_gb = hurricane_df_clean.groupby('Storm_Id')
storm_sgb = storm_gb['Date']
start_date = storm_sgb.min()
end_date = storm_sgb.max()
duration = end_date - start_date


In [9]:
#Merge Start Date, End Date, and Duration to original dataframe. 
start_end_df = pd.DataFrame({"Start Date": start_date
                             ,"End Date": end_date
                             ,"Duration" : duration
                            }).reset_index()

merge_df = pd.merge(hurricane_df_clean, start_end_df, how="outer", on="Storm_Id")

merge_df.head(100)


,Storm_Id,Name,Date,Year,Status,Latitude,Longitude,Windspeed,Pressure,Landfall,Duration,End Date,Start Date
0,AL012003,ANA,2003-04-21 00:00:00,2003,TS,31.4N,66.4W,50,995,,3 days 00:00:00,2003-04-24 00:00:00,2003-04-21 00:00:00
1,AL012003,ANA,2003-04-21 06:00:00,2003,TS,30.6N,65.8W,50,995,,3 days 00:00:00,2003-04-24 00:00:00,2003-04-21 00:00:00
2,AL012003,ANA,2003-04-21 12:00:00,2003,TS,30.0N,64.8W,50,995,,3 days 00:00:00,2003-04-24 00:00:00,2003-04-21 00:00:00
3,AL012003,ANA,2003-04-21 18:00:00,2003,TS,29.4N,63.1W,50,994,,3 days 00:00:00,2003-04-24 00:00:00,2003-04-21 00:00:00
4,AL012003,ANA,2003-04-22 00:00:00,2003,TS,29.4N,61.5W,50,994,,3 days 00:00:00,2003-04-24 00:00:00,2003-04-21 00:00:00
5,AL012003,ANA,2003-04-22 06:00:00,2003,TS,29.5N,60.1W,50,994,,3 days 00:00:00,2003-04-24 00:00:00,2003-04-21 00:00:00
6,AL012003,ANA,2003-04-22 12:00:00,2003,TS,29.6N,58.5W,45,995,,3 days 00:00:00,2003-04-24 00:00:00,2003-04-21 00:00:00
7,AL012003,ANA,2003-04-22 18:00:00,2003,TS,29.7N,56.6W,45,996,,3 days 00:00:00,2003-04-24 00:00:00,2003-04-21 00:00:00
8,AL012003,ANA,2003-04-23 00:00:00,2003,TS,30.0N,54.7W,40,998,,3 days 00:00:00,2003-04-24 00:00:00,2003-04-21 00:00:00
9,AL012003,ANA,2003-04-23 06:00:00,2003,TS,30.6N,53.0W,40,1000,,3 days 00:00:00,2003-04-24 00:00:00,2003-04-21 00:00:00


In [10]:
#Filter only storms that reached Landfall and create dataframe
landfall_df = merge_df.loc[merge_df['Landfall'] == "L"]

In [11]:
#Find the nearest city to the Landfall cooridinates using the Citipy
from citipy import citipy

#strip the Direction from the Latitude and Longitude

lats = landfall_df["Latitude"].str.split("([A-Z]+)", expand=True)
lons = landfall_df["Longitude"].str.split("([A-Z]+)", expand=True)
#grab the number from index 0
lats = lats[0]
lons = lons[0]
#use citipy to find the nearest city
landfall_df.loc[:, "Latitude"] = lats
landfall_df.loc[:, "Longitude"] = lons
#change the column to numeric
landfall_df["Latitude"] = pd.to_numeric(landfall_df["Latitude"])
landfall_df["Longitude"] = pd.to_numeric(landfall_df["Longitude"])
#convert Longitude column to negative
landfall_df["Longitude"] *= -1


#use citipy to find the nearest city
latitude = landfall_df["Latitude"]
longitude = landfall_df["Longitude"]
coordinates = zip(latitude, longitude)
cities = []
for coordinate_pair in coordinates:
    lat, lon = coordinate_pair
    cities.append(citipy.nearest_city(lat,lon))

/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [12]:
# Add city name column to Landfall dataframe
city_name = []
for city in cities:
    name = city.city_name
    city_name.append(name)
landfall_df.loc[:, "Nearest City"] = city_name
landfall_df.head()


    

/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Storm_Id,Name,Date,Year,Status,Latitude,Longitude,Windspeed,Pressure,Landfall,Duration,End Date,Start Date,Nearest City
19,AL032003,BILL,2003-06-30 19:00:00,2003,TS,29.3,-91.0,50,997,L,1 days 12:00:00,2003-07-01 00:00:00,2003-06-29 12:00:00,houma
32,AL042003,CLAUDETTE,2003-07-11 10:00:00,2003,TS,20.8,-86.9,50,1009,L,8 days 00:00:00,2003-07-16 18:00:00,2003-07-08 18:00:00,playa del carmen
50,AL042003,CLAUDETTE,2003-07-15 15:03:00,2003,HU,28.3,-96.2,80,979,L,8 days 00:00:00,2003-07-16 18:00:00,2003-07-08 18:00:00,port lavaca
78,AL082003,ERIKA,2003-08-16 10:03:00,2003,HU,25.3,-97.4,65,986,L,2 days 00:00:00,2003-08-16 18:00:00,2003-08-14 18:00:00,valle hermoso
129,AL112003,GRACE,2003-08-31 11:00:00,2003,TS,29.4,-95.1,35,1007,L,0 days 18:00:00,2003-08-31 12:00:00,2003-08-30 18:00:00,santa fe


In [13]:
# merget landfall and original dataframe 
storm_added_fields = pd.merge(merge_df, landfall_df, how="outer", on="Storm_Id")

storm_added_fields = storm_added_fields.loc[:, ["Storm_Id", "Name_x", "Date_x", "Year_x", "Status_x", "Latitude_x", "Longitude_x"
                                    ,"Windspeed_x", "Pressure_x", "Start Date_x", "End Date_x", "Duration_x", "Landfall_y"
                                    ,"Latitude_y", "Longitude_y", "Nearest City"]]

storm_added_fields_df = storm_added_fields.rename(columns={"Storm_Id": "Storm ID", "Name_x":"Name", "Date_x":"Date", "Year_x": "Year", "Status_x":"Status"
                                                       ,"Latitude_x" : "Max Latitude", "Longitude_x": "Max Longitude"
                                                       ,"Windspeed_x" : "Max Windspeed", "Pressure_x" : "Max Pressure"
                                                       ,"Start Date_x" : "Start Date", "End Date_x" : "End Date", "Duration_x" : "Duration"
                                                       ,"Landfall_y" : "Landfall", "Latitude_y" : "Lf Latitude", "Longitude_y" : "Lf Longitude", "Nearest City": "Nearest City"})


storm_added_fields_df.head()

,Storm ID,Name,Date,Year,Status,Max Latitude,Max Longitude,Max Windspeed,Max Pressure,Start Date,End Date,Duration,Landfall,Lf Latitude,Lf Longitude,Nearest City
0,AL012003,ANA,2003-04-21 00:00:00,2003,TS,31.4N,66.4W,50,995,2003-04-21,2003-04-24,3 days,NaN,NaN,NaN,NaN
1,AL012003,ANA,2003-04-21 06:00:00,2003,TS,30.6N,65.8W,50,995,2003-04-21,2003-04-24,3 days,NaN,NaN,NaN,NaN
2,AL012003,ANA,2003-04-21 12:00:00,2003,TS,30.0N,64.8W,50,995,2003-04-21,2003-04-24,3 days,NaN,NaN,NaN,NaN
3,AL012003,ANA,2003-04-21 18:00:00,2003,TS,29.4N,63.1W,50,994,2003-04-21,2003-04-24,3 days,NaN,NaN,NaN,NaN
4,AL012003,ANA,2003-04-22 00:00:00,2003,TS,29.4N,61.5W,50,994,2003-04-21,2003-04-24,3 days,NaN,NaN,NaN,NaN


In [14]:
#Find row with Max windspeed and return all columns in that row.
clean_storm_df = storm_added_fields_df.iloc[storm_added_fields_df.reset_index().groupby(['Storm ID'])["Max Windspeed"].idxmax()]
clean_storm_df.head()

,Storm ID,Name,Date,Year,Status,Max Latitude,Max Longitude,Max Windspeed,Max Pressure,Start Date,End Date,Duration,Landfall,Lf Latitude,Lf Longitude,Nearest City
0,AL012003,ANA,2003-04-21 00:00:00,2003,TS,31.4N,66.4W,50,995,2003-04-21 00:00:00,2003-04-24 00:00:00,3 days 00:00:00,NaN,NaN,NaN,NaN
375,AL012004,ALEX,2004-08-05 00:00:00,2004,HU,38.5N,66.0W,105,957,2004-08-01 18:00:00,2004-08-06 12:00:00,4 days 18:00:00,NaN,NaN,NaN,NaN
1318,AL012005,ARLENE,2005-06-11 00:00:00,2005,TS,26.5N,85.6W,60,990,2005-06-09 06:00:00,2005-06-11 19:00:00,2 days 13:00:00,L,21.8,-84.5,mantua
2140,AL012006,ALBERTO,2006-06-12 12:00:00,2006,TS,26.8N,86.3W,60,1000,2006-06-11 00:00:00,2006-06-14 00:00:00,3 days 00:00:00,L,29.9,-83.7,tallahassee
2721,AL012008,ARTHUR,2008-05-31 06:00:00,2008,TS,17.8N,88.0W,40,1004,2008-05-31 00:00:00,2008-06-01 06:00:00,1 days 06:00:00,L,17.9,-88.2,san pedro


In [15]:
# Create Bins for each storm category according to https://en.wikipedia.org/wiki/Maximum_sustained_wind
min_wind = clean_storm_df["Max Windspeed"].min()
print(min_wind)
bins = [33, 63, 82, 95, 112, 136, 170]

# Create the names for the four bins
group_names = ['Tropical Storm', 'Category One', 'Category Two', 'Category Three', 'Category Four', 'Category Five']

35


In [16]:
# create new category column
storm_category = pd.cut(clean_storm_df["Max Windspeed"], bins, labels=group_names)
storm_category

0       Tropical Storm
375     Category Three
1318    Tropical Storm
2140    Tropical Storm
2721    Tropical Storm
4020      Category Two
4560    Tropical Storm
5246    Tropical Storm
5876    Tropical Storm
6049      Category Two
6297    Tropical Storm
6549      Category One
7063    Tropical Storm
391     Tropical Storm
1330    Tropical Storm
2444    Tropical Storm
2745    Category Three
3851    Tropical Storm
4567    Tropical Storm
5257    Tropical Storm
5883    Tropical Storm
6306    Tropical Storm
6556    Tropical Storm
7069    Tropical Storm
17      Tropical Storm
476      Category Four
1340      Category One
2152    Tropical Storm
2448    Tropical Storm
2804    Tropical Storm
             ...      
3845     Category Four
4376      Category One
5203      Category One
5795      Category One
8075    Category Three
304     Tropical Storm
1858     Category Five
4393      Category Two
5218    Category Three
5824    Category Three
8081    Tropical Storm
321     Tropical Storm
4429      C

In [17]:
# Add column to clean storm dataframe
clean_storm_df["Storm Category"] = storm_category
clean_storm_df

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Storm ID,Name,Date,Year,Status,Max Latitude,Max Longitude,Max Windspeed,Max Pressure,Start Date,End Date,Duration,Landfall,Lf Latitude,Lf Longitude,Nearest City,Storm Category
0,AL012003,ANA,2003-04-21 00:00:00,2003,TS,31.4N,66.4W,50,995,2003-04-21 00:00:00,2003-04-24 00:00:00,3 days 00:00:00,NaN,NaN,NaN,NaN,Tropical Storm
375,AL012004,ALEX,2004-08-05 00:00:00,2004,HU,38.5N,66.0W,105,957,2004-08-01 18:00:00,2004-08-06 12:00:00,4 days 18:00:00,NaN,NaN,NaN,NaN,Category Three
1318,AL012005,ARLENE,2005-06-11 00:00:00,2005,TS,26.5N,85.6W,60,990,2005-06-09 06:00:00,2005-06-11 19:00:00,2 days 13:00:00,L,21.8,-84.5,mantua,Tropical Storm
2140,AL012006,ALBERTO,2006-06-12 12:00:00,2006,TS,26.8N,86.3W,60,1000,2006-06-11 00:00:00,2006-06-14 00:00:00,3 days 00:00:00,L,29.9,-83.7,tallahassee,Tropical Storm
2721,AL012008,ARTHUR,2008-05-31 06:00:00,2008,TS,17.8N,88.0W,40,1004,2008-05-31 00:00:00,2008-06-01 06:00:00,1 days 06:00:00,L,17.9,-88.2,san pedro,Tropical Storm
4020,AL012010,ALEX,2010-07-01 02:00:00,2010,HU,24.2N,97.7W,95,946,2010-06-26 06:00:00,2010-07-01 18:00:00,5 days 12:00:00,L,17.5,-88.2,belize,Category Two
4560,AL012011,ARLENE,2011-06-30 06:00:00,2011,TS,21.6N,97.0W,55,994,2011-06-28 18:00:00,2011-06-30 18:00:00,2 days 00:00:00,L,21.6,-97.4,tamiahua,Tropical Storm
5246,AL012012,ALBERTO,2012-05-20 00:00:00,2012,TS,32.1N,78.1W,50,995,2012-05-19 12:00:00,2012-05-22 06:00:00,2 days 18:00:00,NaN,NaN,NaN,NaN,Tropical Storm
5876,AL012013,ANDREA,2013-06-06 12:00:00,2013,TS,27.8N,84.9W,55,995,2013-06-05 18:00:00,2013-06-07 12:00:00,1 days 18:00:00,L,29.5,-83.4,lake city,Tropical Storm
6049,AL012014,ARTHUR,2014-07-04 00:00:00,2014,HU,34.0N,77.3W,85,975,2014-07-01 12:00:00,2014-07-05 06:00:00,3 days 18:00:00,L,34.7,-76.6,havelock,Category Two


# Strength of Storm

# Storm Season

# Storm Duration

# Most Dangerous City in relationship to Storms